In [6]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit import Aer, execute
from math import pi

In [7]:
def rotations(qc, reg, n):
    qc.barrier()
    qc.h(reg[n])
    for i in range(n):
        qc.cp(pi/(2**(i+1)), reg[n-(i+1)], reg[n])

def QFT(qc, reg, n):
    for i in range(n+1):
        rotations(qc, reg, n-i)

def addition_state(qc, reg_a, reg_b, n, factor):
    l = len(reg_b)
    for i in range(n+1):
        if (n-i) > l-1:
            pass
        else:
            qc.cp((factor*pi) / (2**i), reg_b[n-i], reg_a[n])
            
def bit_addition(qc, reg_a, reg_b, n, factor):
    for i in range(n+1):
        addition_state(qc, reg_a, reg_b, n-i, factor)
        
def inverseQFTrotations(qc, reg, n):
    for i in range(n):
        qc.cp(-pi/(2**(n-i)), reg[i], reg[n])
    qc.h(reg[n])

def inverseQFT(qc, reg, n):
    for i in range(n+1):
        inverseQFTrotations(qc, reg, i)
        
def add(reg_a, reg_b, qc, factor):
    n = len(reg_a)-1
    
    QFT(qc, reg_a, n)
    bit_addition(qc, reg_a, reg_b, n, factor)
    inverseQFT(qc, reg_a, n)

multiplicand_in = bin(int(input("Enter the multiplicand.")))
multiplicand_in = multiplicand_in[2:]
l1 = len(multiplicand_in)
multiplier_in = bin(int(input("Enter the multiplier.")))
multiplier_in = multiplier_in[2:]
l2 = len(multiplier_in)

if l2 > l1:
    multiplier_in, multiplicand_in = multiplicand_in, multiplier_in
    l2, l1 = l1, l2

multiplicand = QuantumRegister(l1)
multiplier = QuantumRegister(l2)
accumulator = QuantumRegister(l1 + l2)
cl = ClassicalRegister(l1 + l2)
d = QuantumRegister(1)

circ = QuantumCircuit(accumulator, multiplier, multiplicand,
    d, cl, name="qc")

circ.x(d)
# Store bit strings in quantum registers
for i in range(l1):
    if multiplicand_in[i] == '1':
        circ.x(multiplicand[l1 - i - 1])

for i in range(l2):
    if multiplier_in[i] == '1':
        circ.x(multiplier[l1 - i - 1])

multiplier_str = '1'
# Perform repeated addition until the multiplier
# is zero
while(int(multiplier_str) != 0):
    add(accumulator, multiplicand, circ, 1)
    add(multiplier, d, circ, -1)
    for i in range(len(multiplier)):
        circ.measure(multiplier[i], cl[i])
    result = execute(circ, backend=Aer.get_backend('qasm_simulator'),
                    shots=2).result().get_counts(circ.name)
    multiplier_str = list(result.keys())[0]

circ.measure(accumulator, cl)
result = execute(circ, backend=Aer.get_backend('qasm_simulator'),
            shots=100).result().get_counts(circ.name)

print(result)

Enter the multiplicand. 10
Enter the multiplier. 11


{'01101110': 100}
